#### (April 2023 - It works just fine!)

#### Do an HTTP GET request to the Kijiji site, and store the response ✅

#### Parse the response and store a list of objects for which the conditions are met. ✅

#### Send a Telegram message if one or more valid ads have been found. ✅

#### TODO: Handle pagination in the response

#### TODO: Handle non-ASCII characters in the response

In [ ]:
import os
import urllib.request
from bs4 import BeautifulSoup

In [55]:
## Define/retrieve constants

# Telegram
TELEGRAM_TOKEN = os.environ.get('TELEGRAM_TOKEN', '')
TELEGRAM_CHAT_ID = os.environ.get('CHAT_ID', '')
TELEGRAM_URL = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage?chat_id={TELEGRAM_CHAT_ID}"

# Kijiji search
SEARCH_WORD = "canoe"
SEARCH_RADIUS_KM = 100.0 # if few results it will redirect to URL corresponding to 150 km
MAX_PRICE = 300
URL = f"https://www.kijiji.ca/b-dartmouth/{SEARCH_WORD.lower()}/k0l1700109?"\
    f"radius={SEARCH_RADIUS_KM}"
    #f"rb=true&"\
    #f"ll=51.044733%2C-114.071883&"\
    #f"address=Calgary%2C+AB&"\
    # Removing this parameter seems to force the search to the specified radius
    # (otherwise it seems that Kijiji, if few results are found, widens the search to 150 km)
    #f"dc=true" 

# pagination
#https://www.kijiji.ca/b-cars-trucks/calgary/ford-focus/page-2/k0c174l1700199?radius=150.0&address=Calgary%2C+AB&ll=51.044733,-114.071883&rb=true
print(URL)

https://www.kijiji.ca/b-dartmouth/canoe/k0l1700109?radius=100.0


In [44]:
## Do an HTTP GET request to the Kijiji site,
## and store the response

with urllib.request.urlopen(URL) as response:
   
    if response.getcode() == 200: 
        print ("Status OK")
        html_doc= response.read()
        
soup = BeautifulSoup(html_doc, 'html.parser')

Status OK


In [48]:
# "ul" element containing the results
result_list = soup.find("ul", attrs = {"data-testid": "srp-search-list"})
# Each of the "li" children
resultsArray = result_list.find_all("li")

selected_ads = []

for elem in resultsArray:
    
    ad_title = elem.find("h3", attrs = {"data-testid": "listing-title"})
    anchor_elem = ad_title.find("a")
    ad_title_text = ad_title.get_text().strip().replace(" ", "%20")
    
    if "wanted" in ad_title_text.lower():
        print ("Skipping a WANTED ad:", ad_title_text)
        continue
    ad_link = f"https://www.kijiji.ca{anchor_elem.get('href')}"
    
    try:
        ad_price = float(elem.find("p", attrs = {"data-testid": "listing-price"}).get_text().strip().replace("$", "").replace(",", ""))
        ad_description = elem.find("p", attrs = {"data-testid": "listing-description"}).get_text().strip().replace(" ", "%20")
        if (ad_price <= MAX_PRICE):
            selected_ads.append({"title": ad_title_text, "ad_link": ad_link, "ad_price": ad_price, "ad_description": ad_description})
    except:
        # Handling the "Please contact" or no DOM elem for the price
        continue

In [50]:
def composeMessage(item):
    return f"{item.get('title')}%0A{item.get('ad_link')}%0A{item.get('ad_price')}"

In [61]:
def sendTelegramMessage(telegramSendMessageUrl, message):

    with urllib.request.urlopen(telegramSendMessageUrl + f"&text={message}") as response:
        
        if response.getcode() == 200:
            return True

In [62]:
def remove_non_ascii_1(text):
    return ''.join(i for i in text if ord(i)<128)

In [63]:
successful_messages = 0

for raw_message in selected_ads:

    formatted_message = remove_non_ascii_1(composeMessage(raw_message))

    if sendTelegramMessage(TELEGRAM_URL, formatted_message):
        successful_messages += 1
        
print (f"{successful_messages} Telegram messages sent! 😃")

20 Telegram messages sent! 😃
